# 训练 StyleGAN V2 生成草莓图像

在自己的数据集上，训练基于 StyleGAN V2 的 Unconditional GAN（非条件生成对抗网络）。

扩展阅读：

古典花瓶图像生成 https://thisvesseldoesnotexist.com/#/


> 作者：[同济子豪兄](https://space.bilibili.com/1900783)、杨逸飞 2022-4-10

## 进入MMGeneration主目录

In [1]:
import os
os.chdir('mmgeneration')
os.listdir()

['.git',
 '.dev_scripts',
 '.github',
 '.gitignore',
 '.pre-commit-config.yaml',
 '.pylintrc',
 '.readthedocs.yml',
 'CITATION.cff',
 'LICENSE',
 'LICENSES.md',
 'MANIFEST.in',
 'README.md',
 'README_zh-CN.md',
 'apps',
 'configs',
 'demo',
 'docker',
 'docs',
 'mmgen',
 'model-index.yml',
 'requirements.txt',
 'requirements',
 'setup.cfg',
 'setup.py',
 'tests',
 'tools',
 'mmgen.egg-info',
 'outputs',
 'data',
 'checkpoints',
 'work_dirs']

## 下载草莓图像数据集

In [2]:
# 下载草莓图像数据集压缩包
# !wget https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20220322-mmgeneration/watermelon.zip -O data/watermelon.zip
!wget https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20220322-mmgeneration/strawberry.zip -O data/strawberry.zip

--2022-04-10 21:20:07--  https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20220322-mmgeneration/strawberry.zip
Connecting to 172.16.0.13:5848... connected.
Proxy request sent, awaiting response... 200 OK
Length: 43600183 (42M) [application/zip]
Saving to: ‘data/strawberry.zip’

data/strawberry.zip 100%[===================>]  41.58M  43.2MB/s    in 1.0s    

2022-04-10 21:20:09 (43.2 MB/s) - ‘data/strawberry.zip’ saved [43600183/43600183]



In [3]:
# 解压至 data/strawberry 目录
!unzip -o data/strawberry.zip -d data/strawberry

Archive:  data/strawberry.zip
  inflating: data/strawberry/img1.webp  
  inflating: data/strawberry/img2.jpg  
  inflating: data/strawberry/img3.jpg  
  inflating: data/strawberry/img4.jpg  
  inflating: data/strawberry/img5.jpg  
  inflating: data/strawberry/img6.jpg  
  inflating: data/strawberry/img7.webp  
  inflating: data/strawberry/img8.jpg  
  inflating: data/strawberry/img9.jpg  
  inflating: data/strawberry/img10.jpg  
  inflating: data/strawberry/img11.jpg  
  inflating: data/strawberry/img12.jpg  
  inflating: data/strawberry/img13.jpg  
  inflating: data/strawberry/img14.jpg  
  inflating: data/strawberry/img15.jpg  
  inflating: data/strawberry/img16.jpg  
  inflating: data/strawberry/img17.jpg  
  inflating: data/strawberry/img18.jpg  
  inflating: data/strawberry/img19.jpg  
  inflating: data/strawberry/img20.jpg  
  inflating: data/strawberry/img21.jpg  
  inflating: data/strawberry/img22.jpg  
  inflating: data/strawberry/img23.jpg  
  inflating: data/strawberry/img24

### 训练用于 FID 评估指标的 Inception V3 模型

生成 `work_dirs/inception_pkl/strawberry.pkl` 文件

In [4]:
!python tools/utils/inception_stat.py \
        --imgsdir data/strawberry \
        --pklname strawberry.pkl \
        --size 256 \
        --flip \
        --num-samples -1

2022-04-10 21:20:32,991 - mmgen - INFO - dataset_name: <class 'mmgen.datasets.unconditional_image_dataset.UnconditionalImageDataset'>, total 889 images in imgs_root: data/strawberry
/environment/miniconda3/lib/python3.7/site-packages/torchvision/models/inception.py:83: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  ' due to scipy/scipy#11299), please set init_weights=True.', FutureWarning)
Downloading: "https://github.com/mseitzer/pytorch-fid/releases/download/fid_weights/pt_inception-2015-12-05-6726825d.pth" to /home/featurize/.cache/torch/hub/checkpoints/pt_inception-2015-12-05-6726825d.pth
100%|██████████████████████████████████████| 91.2M/91.2M [00:01<00:00, 92.1MB/s]
2022-04-10 21:20:41,429 - mmgen - INFO - Use all samples in subset
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 36/36, 12.

### 配置 config 文件

In [5]:
!wget https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20220322-mmgeneration/stylegan2_c2_ffhq_256_b4x8_800k_strawberry.py -O configs/styleganv2/stylegan2_c2_ffhq_256_b4x8_800k_strawberry.py

--2022-04-10 21:21:16--  https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20220322-mmgeneration/stylegan2_c2_ffhq_256_b4x8_800k_strawberry.py
Connecting to 172.16.0.13:5848... connected.
Proxy request sent, awaiting response... 200 OK
Length: 1764 (1.7K) [binary/octet-stream]
Saving to: ‘configs/styleganv2/stylegan2_c2_ffhq_256_b4x8_800k_strawberry.py’

configs/styleganv2/ 100%[===================>]   1.72K  --.-KB/s    in 0s      

2022-04-10 21:21:21 (43.3 MB/s) - ‘configs/styleganv2/stylegan2_c2_ffhq_256_b4x8_800k_strawberry.py’ saved [1764/1764]



## 在自己的数据集上训练模型(运行半小时左右)

训练过程中，在 work_dirs/experiments/experiments_name目录下

training_samples保存了训练过程中达到不同迭代次数时的训练效果

ckpt/experiments_name保存了训练过程中达到不同迭代次数时的模型权重文件

In [8]:
!bash tools/dist_train.sh configs/styleganv2/stylegan2_c2_ffhq_256_b4x8_800k_strawberry.py 1 --work-dir work_dirs/experiments/stylegan2_c2_ffhq_256_b4x8_800k

/environment/miniconda3/lib/python3.7/site-packages/torch/distributed/launch.py:186: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  FutureWarning,
tools/train.py:98: UserWarning: Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  f'Setting OMP_NUM_THREADS environment variable for each process '
tools/train.py:108: UserWarning: Setting MKL_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performan